# セグメント木



## セグメント木

蟻本p.153

https://ikatakos.com/pot/programming_algorithm/data_structure/segment_tree

セグメント木 (Segment Tree) は要素列を表現するデータ構造の1つ。  
要素列 $A = a_1 a_2 a_3 \ldots a_n$ に対して、区間 $A_{[l, r)}$ に対する操作を高速に行うことができる。

## Range Minimum Query (RMQ)

セグメント木の用途の1つ。  
要素の更新に $\mathcal{O}(\log{n})$ 費やす代わりに、区間 $[l, r)$ の最小値を求める問い合わせを $\mathcal{O}(\log{n})$ で行うことができる。  

セグメント木の実装を示す。操作用のメソッドは0-indexedだが、データを格納する配列は1-indexedとしていることに注意。  
問い合わせ操作には分割統治法を用いている。

In [1]:
class SegmentTree():

    def __init__(self, size, op=min, init_value=10**8):
        """初期化"""
        self.size = size
        self.op = op
        self.init_value = init_value
        n = 2 ** ((size-1).bit_length())
        treesize = n * 2
        st = [init_value for i in range(treesize)]
        self.st = st
    
    def update(self, key, value):
        """値の更新"""
        offset = len(self.st) // 2
        k = offset + key
        self.st[k] = value
        k >>= 1
        while k > 0:
            self.st[k] = self.op(self.st[k * 2], self.st[k * 2 + 1])
            k >>= 1
    
    @classmethod
    def from_array(cls, a, op=min, init_value=10**8):
        st = cls(len(a), op=op, init_value=init_value)
        for i, x in enumerate(a):
            st.update(i, x)
        return st
    
    def _query(self, a, b, k, l, r):
        """区間[a, b) に対する操作
        k: 着目しているノード (1-indexed)
        l: 探索区間 st[l, r) の左端 (0-indexed)
        r: 探索区間 st[l, r) の右端 (0-indexed)
        """
        if r <= a or b <= l:
            return self.init_value
        if a <= l and r <= b:
            return self.st[k]
        mid = (l + r) // 2
        lv = self._query(a, b, k * 2, l, mid)
        rv = self._query(a, b, k * 2 + 1, mid, r)
        return self.op(lv, rv)

    def query(self, a, b):
        """区間[a, b) に対する操作"""
        if a > b:
            raise ValueError("a must be less than equal b.")
        return self._query(a, b, k=1, l=0, r=len(self.st)//2)

In [2]:
A = [3, -5, 2, -10, 1, 4, 11]

st = SegmentTree.from_array(A)

a = 3
b = 7
print(A[a:b])
print('{}'.format(st.query(a, b)))

[-10, 1, 4, 11]
-10


セグメント木による区間に対する操作は、最小値以外の操作でも利用できる。  
具体的には、次の条件を満たせばよい。

- 結合法則が成り立つ $(a \cdot b) \cdot c = a \cdot (b \cdot c)$
- 単位元 $e$ をもつ $a \cdot e = e \cdot a = a$

操作 `op` には二項演算子を、初期値 `init_value` には単位元を指定する。

| クエリ | 操作 | 初期値 | 補足 |
|:--|:--|:--:|:--|
| 和 | `operator.add` | 0 | 累積和を使えば $\mathcal{O}(1)$ |
| 積 | `operator.mul` | 1 | 累積積を使えば $\mathcal{O}(1)$ |
| 最小値 | `min` | +INF | |
| 最大値 | `max` | -INF | |
| AND | `operator.and` | 1 | |
| OR | `operator.and` | 0 | |
| XOR | `operator.xor` | 0 | |
| GCD | `math.gcd` | 0 | $\gcd(x, 0) = x$ |
| LCM | &#x2015; | 1 | $\operatorname{lcm}(x, 1) = x$ |